In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Supervised Learning: Classification

## Reading and investigating the data

In [ ]:
data = pd.read_csv('bankdata.csv')

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.head()

In [ ]:
data['status'].value_counts()

### Investigating the numerical variables

In [ ]:
data.describe()

#### Investigating variable duration

In [ ]:
data['duration'].value_counts()

In [ ]:
# we will convert this to a categorical
data['duration'] = data['duration'].apply(str)

In [ ]:
# Checking for multicollinearity
plt.figure(figsize=(10, 8))
ax = sns.heatmap(data.corr(), annot=True)
plt.show()

payments and amount_left have a rather high correlation, so we will include only one of them in the model

In [ ]:
#sns.distplot(data['amount_total'])
#plt.show()

#sns.distplot(data['balance'])
#plt.show()

sns.distplot(data['payments'])
plt.show()

### Preprocessing the numerical variables

In [ ]:
from sklearn.preprocessing import StandardScaler
data_num = data.select_dtypes(include = np.number).drop(columns = 'amount_left')
print(data_num.head())
# Standardizing data
transformer = StandardScaler().fit(data_num)
data_num_standardized = transformer.transform(data_num)
x = pd.DataFrame(data_num_standardized, columns = data_num.columns)
print(x.head())

In [ ]:
sns.distplot(data['balance'])
plt.show()

In [ ]:
sns.distplot(x['balance'])
plt.show()

### Getting dummies for the categorical variables

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat = data.select_dtypes(include = np.object).drop(columns = 'status')
enc = OneHotEncoder()
categorical = pd.DataFrame(enc.fit_transform(cat).toarray(), columns = enc.get_feature_names())
categorical.head()

In [ ]:
categorical.sum(axis = 1)

In [ ]:
y = data['status']
X = pd.concat((x, categorical), axis=1)
X.head()

#### Splitting into train and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticr = LogisticRegression(max_iter = 500).fit(X_train, y_train)

In [ ]:
predictions = logisticr.predict(X_test)
logisticr.score(X_test, y_test)

In [ ]:
y_test.value_counts()

In [ ]:
pd.Series(predictions).value_counts()

What happened in terms of the classes?

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
predictions = decisiontree.predict(X_test)
decisiontree.score(X_test, y_test)

In [ ]:
pd.Series(predictions).value_counts()

In [ ]:
decisiontree.feature_importances_

In [ ]:
print(dict(zip(X.columns, decisiontree.feature_importances_)))

In [ ]:
plt.rcParams['figure.figsize'] = [25, 6]
plt.bar(height = decisiontree.feature_importances_, x = X.columns)
plt.show()

### SVM

In [ ]:
from sklearn.svm import SVC

svm = SVC().fit(X_train, y_train)
predictions = svm.predict(X_test)
print(svm.score(X_test, y_test))
print(pd.Series(predictions).value_counts())

### K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

nearest_neighbors = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
predictions = nearest_neighbors.predict(X_test)
print(nearest_neighbors.score(X_test, y_test))
print(pd.Series(predictions).value_counts())

In [ ]:
help(KNeighborsClassifier)

## Supervised Learning: Regression

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
# SVR
from sklearn.svm import SVR
# KNN
from sklearn.neighbors import KNeighborsRegressor

## Model Evaluation

### Creating the Confusion Matrix

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticr = LogisticRegression(max_iter = 500).fit(X_train, y_train)

In [ ]:
predictions = logisticr.predict(X_test)
logisticr.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, predictions)
print(cf_matrix)

In [ ]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

#### Precision, Recall and the F1 Score

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
# We need to assign which is the 'positive class'. Here, we are trying to predict whether a customer will run into
# problems repaying their loan. So Status B is the 'positive' class (meaning credit problems = True)
print(precision_score(y_test, predictions, pos_label='B'))
print(recall_score(y_test, predictions, pos_label='B'))
print(f1_score(y_test, predictions, pos_label='B'))

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

#### ROC and AUC analysis

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
help(roc_curve)

In [ ]:
# we need to calculate class probabilities!
y_pred_proba = logisticr.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba, pos_label='B') 
plt.plot(fpr,tpr)

In [ ]:
help(roc_auc_score)

In [ ]:
roc_auc_score(y_test, y_pred_proba)

### Repeating all the steps for a Decision Tree for comparison:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
predictions = decisiontree.predict(X_test)
decisiontree.score(X_test, y_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, predictions)
print(cf_matrix)

In [ ]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
print(precision_score(y_test, predictions, pos_label='B'))
print(recall_score(y_test, predictions, pos_label='B'))
print(f1_score(y_test, predictions, pos_label='B'))

In [ ]:
balanced_accuracy_score(y_test, predictions)

In [ ]:
y_pred_proba = decisiontree.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba, pos_label='B') 
# we need to assign which one is the "positive" (1) category
plt.plot(fpr,tpr)

In [ ]:
roc_auc_score(y_test, y_pred_proba)

### Hyperparameter Tuning

Can we do even better?

In [ ]:
from sklearn.model_selection import GridSearchCV
help(GridSearchCV)

In [ ]:
help(DecisionTreeClassifier)

In [ ]:
param_grid_rf = {"criterion": ["gini", "entropy"],
                 "splitter": ["best", "random"],
                 "max_depth": [3, 10, None]}

tune_rf = GridSearchCV(DecisionTreeClassifier(),
                       param_grid=param_grid_rf,
                       cv=3)

tune_rf.fit(X_train, y_train)

In [ ]:
tune_rf.best_params_

In [ ]:
decisiontree_opt = DecisionTreeClassifier(**tune_rf.best_params_).fit(X_train, y_train)

In [ ]:
predictions = decisiontree_opt.predict(X_test)
decisiontree_opt.score(X_test, y_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, predictions)
print(cf_matrix)

In [ ]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
print(precision_score(y_test, predictions, pos_label='B'))
print(recall_score(y_test, predictions, pos_label='B'))
print(f1_score(y_test, predictions, pos_label='B'))

In [ ]:
balanced_accuracy_score(y_test, predictions)

### Cross Validation

Is our split into training and test set biased?

In [ ]:
from sklearn.model_selection import cross_validate
help(cross_validate)

In [ ]:
cross_validate(DecisionTreeClassifier(),
               X_train,
               y_train,
               scoring="balanced_accuracy")

In [ ]:
cross_validate(DecisionTreeClassifier(),
               X_train,
               y_train,
               scoring="balanced_accuracy")["test_score"].mean()